In [1]:
!pip install wandb pytorch-lightning transformers[sentencepiece] onnx

In [14]:
import wandb
import pytorch_lightning as pl
import torch
from transformers import BertModel, BertConfig
from torchmetrics import Metric
import os
import onnx
import onnx.numpy_helper as numpy_helper
import json
import numpy as np

In [3]:
run = wandb.init()
artifact = run.use_artifact('prince_/lit-wandb/model.ckpt:v34', type='model')
artifact_dir = artifact.download()

wandb: Currently logged in as: prince_. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model.ckpt:v34, 417.81MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.1


In [4]:
run.use_artifact??

Read the documentation on hot to save the model in torch

In [15]:
pl.LightningModule.to_torchscript??

In [5]:
path = "/content/drive/MyDrive/model/B_data.json"
with open(path, 'r') as f:
    dataset = json.load(f)

path = "/content/drive/MyDrive/model/training_set.json"
with open(path, 'r') as f:
    test_dataset = json.load(f)

text, intent, ner = [], [], []
for i in dataset:
    text.append(i['text'])
    intent.append(i['intent'])
    ner.append(i['entities'].split())

test_text, test_intent, test_ner = [], [], []
for i in test_dataset:
    test_text.append(i['text'])
    test_intent.append(i['intent'])
    test_ner.append(i['entities'].split())

unique_intents = set(intent)
num_intent_labels = len(unique_intents)

one_dimensional_ner = [tag for subset in ner for tag in subset ]
unique_ner = set(one_dimensional_ner)
num_ner_labels = len(unique_ner)



In [17]:
# num_ner_labels, num_intent_labels = 9, 5

In [6]:
class MyAccuracy(Metric):
    def __init__(self):
        super().__init__()
        self.add_state('total', default=torch.tensor(0), dist_reduce_fx='sum')
        self.add_state('correct', default=torch.tensor(0), dist_reduce_fx='sum')

    def update(self, logits, labels, num_labels):

        flattened_targets = labels.view(-1) # shape (batch_size, sequence_len)
        active_logits = logits.view(-1, num_labels) # shape (batch_size * sequence_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * sequence_len)

        # compute accuracy only at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, sequnce_len)
        ac_labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        self.correct += torch.sum(ac_labels == predictions)
        self.total += torch.numel(ac_labels)

    def compute(self):
        return self.correct.float() / self.total.float()

In [7]:
class MultiTaskBertModel(pl.LightningModule):

    """
    Multi-task Bert model for Named Entity Recognition (NER) and Intent Classification

    Args:
        config (BertConfig): Bert model configuration.
        num_ner_labels (int): The number of labels for NER task.
        num_intent_labels (int): The number of labels for Intent Classification task.
    """

    def __init__(self, config, num_ner_labels, num_intent_labels):
        super().__init__()

        self.num_ner_labels = num_ner_labels
        self.num_intent_labels = num_intent_labels

        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.model = BertModel(config=config)

        self.ner_classifier = torch.nn.Linear(config.hidden_size, self.num_ner_labels)
        self.intent_classifier = torch.nn.Linear(config.hidden_size, self.num_intent_labels)

        # log hyperparameters
        self.save_hyperparameters()

        self.accuracy = MyAccuracy()

    def forward(self, input_ids=None, attention_mask=None):

        """
        Perform a forward pass through Multi-task Bert model.

        Args:
            input_ids (torch.Tensor): Input token IDs.
            attention_mask (torch.Tensor): Attention mask for input tokens.
            ner_labels (torch.Tensor): Labels for NER task.
            intent_labels (torch.Tensor): Labels for Intent Classification task.

        Returns:
            Tuple[torch.Tensor,torch.Tensor,torch.Tensor,torch.Tensor]: NER loss, NER logits, Intent loss, Intent logits.

        Raises:
            ValueError: If ner_labels or intent_labels were not provided.
        """

        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        ner_logits = self.ner_classifier(sequence_output)

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        intent_logits = self.intent_classifier(pooled_output)

        return ner_logits, intent_logits

    def training_step(self: pl.LightningModule, batch, batch_idx: int):
        loss, ner_logits, intent_logits, ner_labels, intent_labels = self._common_step(batch, batch_idx)
        accuracy_ner = self.accuracy(ner_logits, ner_labels, num_ner_labels)
        accuracy_intent = self.accuracy(intent_logits, intent_labels, num_intent_labels)
        self.log_dict({'training_loss': loss, 'ner_accuracy': accuracy_ner, 'intent_accuracy': accuracy_intent},
                      on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def on_validation_epoch_start(self):
        self.validation_step_outputs_ner = []
        self.validation_step_outputs_intent = []

    def validation_step(self, batch, batch_idx):
        loss, ner_logits, intent_logits, ner_labels, intent_labels = self._common_step(batch, batch_idx)
        # self.log('val_loss', loss)
        accuracy_ner = self.accuracy(ner_logits, ner_labels, num_ner_labels)
        accuracy_intent = self.accuracy(intent_logits, intent_labels, num_intent_labels)
        self.log_dict({'validation_loss': loss, 'val_ner_accuracy': accuracy_ner, 'val_intent_accuracy': accuracy_intent},
                      on_step=False, on_epoch=True, prog_bar=True)

        self.validation_step_outputs_ner.append(ner_logits)
        self.validation_step_outputs_intent.append(intent_logits)
        return loss

    def on_validation_epoch_end(self):
        validation_step_outputs_ner = self.validation_step_outputs_ner
        validation_step_outputs_intent = self.validation_step_outputs_intent

        dummy_input = torch.zeros((1, 128), device=self.device, dtype=torch.long)
        model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
        torch.onnx.export(self, dummy_input, model_filename)
        artifact = wandb.Artifact(name="model.ckpt", type="model")
        artifact.add_file(model_filename)
        self.logger.experiment.log_artifact(artifact)

        flattened_logits_ner = torch.flatten(torch.cat(validation_step_outputs_ner))
        flattened_logits_intent = torch.flatten(torch.cat(validation_step_outputs_intent))
        self.logger.experiment.log(
            {"valid/ner_logits": wandb.Histogram(flattened_logits_ner.to('cpu')),
             "valid/intent_logits": wandb.Histogram(flattened_logits_intent.to('cpu')),
             "global_step": self.global_step}
        )

    def _common_step(self, batch, batch_idx):
        ids = batch['input_ids']
        mask = batch['attention_mask']
        ner_labels = batch['ner_labels']
        intent_labels = batch['intent_labels']

        ner_logits, intent_logits = self.forward(ids, mask)

        criterion = torch.nn.CrossEntropyLoss()

        ner_loss = criterion(ner_logits.view(-1, self.num_ner_labels), ner_labels.view(-1))
        intent_loss = criterion(intent_logits.view(-1, self.num_intent_labels), intent_labels.view(-1))

        loss = ner_loss + intent_loss
        return loss, ner_logits, intent_logits, ner_labels, intent_labels

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        return optimizer

In [8]:
config = BertConfig.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
model = MultiTaskBertModel(config, num_ner_labels, num_intent_labels)

In [19]:
total_params = 0
for name, param in model.named_parameters():
    total_params += param.numel()

print("Total number of parameters in the model:", total_params)

Total number of parameters in the model: 109493006


In [10]:
onnx_model = onnx.load(os.path.join(artifact_dir, "model_00280.onnx"))

In [17]:
state_dict = {}

for initializer in onnx_model.graph.initializer:
    # Convert the initializer's data to a PyTorch tensor
    tensor = numpy_helper.to_array(initializer)

    # Use the initializer's name as the key in the state_dict
    state_dict[initializer.name] = torch.tensor(tensor)

In [22]:
total_params = sum(p.numel() for p in state_dict.values())
total_params

109493006

In [ ]:
state_dict

In [12]:
model.load_state_dict(state_dict)

RuntimeError: Error(s) in loading state_dict for MultiTaskBertModel:
	Missing key(s) in state_dict: "model.encoder.layer.0.attention.self.query.weight", "model.encoder.layer.0.attention.self.key.weight", "model.encoder.layer.0.attention.self.value.weight", "model.encoder.layer.0.attention.output.dense.weight", "model.encoder.layer.0.intermediate.dense.weight", "model.encoder.layer.0.output.dense.weight", "model.encoder.layer.1.attention.self.query.weight", "model.encoder.layer.1.attention.self.key.weight", "model.encoder.layer.1.attention.self.value.weight", "model.encoder.layer.1.attention.output.dense.weight", "model.encoder.layer.1.intermediate.dense.weight", "model.encoder.layer.1.output.dense.weight", "model.encoder.layer.2.attention.self.query.weight", "model.encoder.layer.2.attention.self.key.weight", "model.encoder.layer.2.attention.self.value.weight", "model.encoder.layer.2.attention.output.dense.weight", "model.encoder.layer.2.intermediate.dense.weight", "model.encoder.layer.

In [ ]:
scripted_model = model.to_torchscript(file_path=artifact_dir)

In [ ]:
# Step 1: Save the TorchScript model to a .pt file
torch.jit.save(model, "model.pt")

# Step 2: Log the .pt file as an artifact
run = wandb.init(project="prince", job_type="model_upload")
artifact = wandb.Artifact("model", type="model")
artifact.add_file("model.pt")
run.log_artifact(artifact)

# Step 3: Finish the run
run.finish()